# 1. Introdução


Para resolver o desfio, estarei utilizando este notebook para delinear as etapas percorridas. Primeiramente, vamos analisar o que deve ser feito:



```
1.   Ler o arquivo gotham_op.txt;
2.   Retornar o total que foi comprado e vendido no dia por ativo (TICKER)
        * Somar as compras (BUY) e subtrair as vendas (SELL) das linhas válidas agrupando pelo TICKER;
3. Informar quais linhas são inválidas.
```
Com isso, vamos analisar o arquivo `gotham_op.txt`:

In [1]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [2]:
g = open('/drive/My Drive/gotham_op.txt', 'r')
print(g.read())
g.close()

SIDE:BUY;QTY:100;TICKER:ACEC4
SIDE:BUY;QTY:1000;TICKER:WAYN3
SIDE:SELL;QTY:369;TICKER:ACEC4
SIDE:SELL;QTY:200;TICKER:WAYN3
SIDE:BUY;QTY:1000;TICKER:LEXC0
SIDE:SELL;QTY:10;TICKER:ACEC4
SIDE:BUY;QTY:5000;TICKER:WAYN3
SIDE:BUY;QTY:100;TICKER:ACE4
SIDE:B;QTY:100;TICKER:QEEN36
SIDE:SELL;QTY:2;TICKER:ACEC4
SIDE:SELL;QTY:100;TICKER:QEEN36
SIDE:BUY;QTY:-100;TICKER:ACEC4
SIDE:SELL;QTY:1200;TICKER:WAYN3
SIDE:SELL;QTY:100;TICKER:LEXC0
SIDE:SELL;QTY:9;TICKER:ACEC4
SIDE:BUY;QTY:200;TICKER:ACEC43
SIDE:SELL;QTY:900;TICKER:QEEN36
SIDE:BUY;QTY:1;TICKER:ACEC4
SIDE:BUY;QTY:2000;TICKER:LEXC0
SIDE:BUY;QTY:100;TICKER:ACEC4
SIDE:SELL;QTY:-100;TICKER:ACE4
SIDE:BUY;QTY:200;TICKER:ACEC43
SIDE:SELL;QTY:100;TICKER:WAYN3
SIDE:BUY;QTY:100;TICKER:ACEC4
SIDE:BUY;QTY:100;TICKER:ACEC4
SIDE:BUY;QTY:100;TICKER:ACECHEMICAL
SIDE:B;QTY:100;TICKER:WAYNE
SIDE:S;QTY:-100;TICKER:ACE4
SIDE:SELL;QTY:900;TICKER:QEEN36
SIDE:BUY;QTY:100;TICKER:DAILYPLANET
SIDE:S;QTY:100;TICKER:WAYNE
SIDE:SELL;QTY:SELL;TICKER:ACE4
SIDE:BUY;QTY:100;TI

Como vemos, cada linha de texto se assemelha a um dict, na forma:

`SIDE:{side_type};QTY:{quantity};TICKER:{ticker}`

Definidos pelas regras:

    * side_type: representa se foi uma compra ou venda
        uma string que pode ter o valor BUY ou SELL
    * quantity: representa a quantidade de ativos que foi comprada ou vendida
        um número inteiro positivo não nulo
        o número é multiplo de 10
    * ticker: representa o ativo
        uma string de cinco a seis caracteres
        os quatro primeiros caracteres são sempre letras
        quando tem somente 5 caracteres o último é um número de 0 a 9 (e.g.: WAYN3, PALM9)
        quando tem 6 caracteres os dois últimos são números de 0 a 9 (e.g.: ACME11, QEEN36)


# 2. Separação de linhas inválidas

Olhando superficialmente para o texto, já podemos perceber que existem linhas inválidas. Para analisá-las, vamos utilizar o módulo Regular Expressions, ou `regex`.

In [3]:
import re

Para separar as linhas válidas das inválidas, vamos primeiro criar duas listas vazias para preencher com os resultados:

In [4]:
validas = []
invalidas = []

Vamos começar nossa classificação utilizando `SIDE`. Vamos analisar os valores deste parâmetro utilizando o método `.split():`

In [5]:
with open('/drive/My Drive/gotham_op.txt', 'r') as gotham:
  print('SIDE: ')
  for linha in gotham:
    print(linha.split(';')[0].split(':')[-1], end = ' ')

SIDE: 
BUY BUY SELL SELL BUY SELL BUY BUY B SELL SELL BUY SELL SELL SELL BUY SELL BUY BUY BUY SELL BUY SELL BUY BUY BUY B S SELL BUY S SELL BUY BUY BUY SELL BUY BUY BUY SELL BUY BUY SELL BUY SELL BUY SELL SELL BUY BUY BUY BUY SELL S SELL BUY BUY BUY SELL SELL BUY SELL SELL BUY B BUY BUY SELL BUY BUY BUY SELL BUY B BUY BUY 

Podemos ver que os únicos valores inválidos são `B` e `S`. Criaremos então uma função para detectar valores inválidos de SIDE em uma linha. Para separar os valores, vamos utilizar o padrão regex `r'/w{3,}'`, que procura uma string de três ou mais caracteres dentro do texto:

In [6]:
def side_invalido(linha):
  s = linha.split(';')[0].split(':')[-1]
  if not re.search(r'\w{3,}', s):
    invalidas.append(linha)

In [7]:
with open('/drive/My Drive/gotham_op.txt', 'r') as gotham:
    for linha in gotham:
      side_invalido(linha)

In [8]:
invalidas

['SIDE:B;QTY:100;TICKER:QEEN36\n',
 'SIDE:B;QTY:100;TICKER:WAYNE\n',
 'SIDE:S;QTY:-100;TICKER:ACE4\n',
 'SIDE:S;QTY:100;TICKER:WAYNE\n',
 'SIDE:S;QTY:100;TICKER:LEXC0\n',
 'SIDE:B;QTY:81;TICKER:ACEC4\n',
 'SIDE:B;QTY:100;TICKER:LEX4\n']

Agora que separamos as linhas por `SIDE`, vamos repetir este processo com `QTY` e `TICKER`:

In [9]:
with open('/drive/My Drive/gotham_op.txt', 'r') as gotham:
  print('QTY: ')
  for linha in gotham:
    print(linha.split(';')[1].split(':')[-1], end = ' ')

QTY: 
100 1000 369 200 1000 10 5000 100 100 2 100 -100 1200 100 9 200 900 1 2000 100 -100 200 100 100 100 100 100 -100 900 100 100 SELL 100 100 100 SELL 100 7000 300 100 200 100 SELL 100 100 50 369 200 33 -1 2000 100 1200 100 10 70 100 95 2 100 -100 9 100 100 81 100 100 SELL 100 650 300 100 200 100 42 100 

In [10]:
def qty_invalido(linha):
  q = linha.split(';')[1].split(':')[-1]
  # Padrão definindo um número com um ou mais dígitos, terminando em 0, junto com condição para não repetir linhas:
  if not re.search(r'^[\d]+[0]$', q) and linha not in invalidas:
    invalidas.append(linha)

In [11]:
with open('/drive/My Drive/gotham_op.txt', 'r') as gotham:
    for linha in gotham:
      qty_invalido(linha)

In [12]:
invalidas

['SIDE:B;QTY:100;TICKER:QEEN36\n',
 'SIDE:B;QTY:100;TICKER:WAYNE\n',
 'SIDE:S;QTY:-100;TICKER:ACE4\n',
 'SIDE:S;QTY:100;TICKER:WAYNE\n',
 'SIDE:S;QTY:100;TICKER:LEXC0\n',
 'SIDE:B;QTY:81;TICKER:ACEC4\n',
 'SIDE:B;QTY:100;TICKER:LEX4\n',
 'SIDE:SELL;QTY:369;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:2;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:-100;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:9;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:1;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:-100;TICKER:ACE4\n',
 'SIDE:SELL;QTY:SELL;TICKER:ACE4\n',
 'SIDE:BUY;QTY:33;TICKER:LEXC0\n',
 'SIDE:BUY;QTY:-1;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:95;TICKER:QEEN36\n',
 'SIDE:BUY;QTY:-100;TICKER:ACECHEMICAL\n',
 'SIDE:BUY;QTY:42;TICKER:ACE43\n']

In [13]:
with open('/drive/My Drive/gotham_op.txt', 'r') as gotham:
  print('TICKER: ')
  for linha in gotham:
    print(linha.split(';')[2].split(':')[-1][:-2], end = ' ')

TICKER: 
ACEC WAYN ACEC WAYN LEXC ACEC WAYN ACE QEEN3 ACEC QEEN3 ACEC WAYN LEXC ACEC ACEC4 QEEN3 ACEC LEXC ACEC ACE ACEC4 WAYN ACEC ACEC ACECHEMICA WAYN ACE QEEN3 DAILYPLANE WAYN ACE LEX WAY ACE ACE DAILYPLANE ACME1 PALM ACEC ACE4 ACE ACE ACEC WAYN WAYN ACEC WAYN LEXC ACEC LEXC ACEC LEXC4 LEXC ACEC WAYN ACE QEEN3 ACEC QEEN3 ACECHEMICA ACEC WAYN ACEC ACEC ACECHEMICA WAYN ACE DAILYPLANE ACME1 PALM ACEC ACE4 LEX ACE4 ACE 

In [14]:
def ticker_invalido(linha):
  t = linha.split(';')[2].split(':')[-1]
  # Padrão definindo 4 letras, junto com um ou mais números:
  if not re.search(r'\D{4,}\d+', t) and linha not in invalidas:
    invalidas.append(linha)

In [15]:
with open('/drive/My Drive/gotham_op.txt', 'r') as gotham:
    for linha in gotham:
      ticker_invalido(linha)

In [16]:
invalidas

['SIDE:B;QTY:100;TICKER:QEEN36\n',
 'SIDE:B;QTY:100;TICKER:WAYNE\n',
 'SIDE:S;QTY:-100;TICKER:ACE4\n',
 'SIDE:S;QTY:100;TICKER:WAYNE\n',
 'SIDE:S;QTY:100;TICKER:LEXC0\n',
 'SIDE:B;QTY:81;TICKER:ACEC4\n',
 'SIDE:B;QTY:100;TICKER:LEX4\n',
 'SIDE:SELL;QTY:369;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:2;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:-100;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:9;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:1;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:-100;TICKER:ACE4\n',
 'SIDE:SELL;QTY:SELL;TICKER:ACE4\n',
 'SIDE:BUY;QTY:33;TICKER:LEXC0\n',
 'SIDE:BUY;QTY:-1;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:95;TICKER:QEEN36\n',
 'SIDE:BUY;QTY:-100;TICKER:ACECHEMICAL\n',
 'SIDE:BUY;QTY:42;TICKER:ACE43\n',
 'SIDE:BUY;QTY:100;TICKER:ACE4\n',
 'SIDE:BUY;QTY:100;TICKER:ACECHEMICAL\n',
 'SIDE:BUY;QTY:100;TICKER:DAILYPLANET\n',
 'SIDE:BUY;QTY:100;TICKER:LEX4\n',
 'SIDE:BUY;QTY:100;TICKER:WAY3\n',
 'SIDE:BUY;QTY:200;TICKER:ACE43\n',
 'SIDE:SELL;QTY:100;TICKER:WAYNE\n',
 'SIDE:BUY;QTY:100;TICKER:WAYNE\n']

Agora que temos nossa lista de linhas inválidas, podemos definir as válidas:

In [17]:
with open('/drive/My Drive/gotham_op.txt', 'r') as gotham:
    for linha in gotham:
      if linha not in invalidas:
        validas.append(linha)

In [18]:
validas

['SIDE:BUY;QTY:100;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:1000;TICKER:WAYN3\n',
 'SIDE:SELL;QTY:200;TICKER:WAYN3\n',
 'SIDE:BUY;QTY:1000;TICKER:LEXC0\n',
 'SIDE:SELL;QTY:10;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:5000;TICKER:WAYN3\n',
 'SIDE:SELL;QTY:100;TICKER:QEEN36\n',
 'SIDE:SELL;QTY:1200;TICKER:WAYN3\n',
 'SIDE:SELL;QTY:100;TICKER:LEXC0\n',
 'SIDE:BUY;QTY:200;TICKER:ACEC43\n',
 'SIDE:SELL;QTY:900;TICKER:QEEN36\n',
 'SIDE:BUY;QTY:2000;TICKER:LEXC0\n',
 'SIDE:BUY;QTY:100;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:200;TICKER:ACEC43\n',
 'SIDE:SELL;QTY:100;TICKER:WAYN3\n',
 'SIDE:BUY;QTY:100;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:100;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:900;TICKER:QEEN36\n',
 'SIDE:BUY;QTY:7000;TICKER:ACME11\n',
 'SIDE:BUY;QTY:300;TICKER:PALM9\n',
 'SIDE:SELL;QTY:100;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:100;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:50;TICKER:WAYN3\n',
 'SIDE:SELL;QTY:200;TICKER:WAYN3\n',
 'SIDE:BUY;QTY:2000;TICKER:LEXC0\n',
 'SIDE:BUY;QTY:100;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:1200;TICKER:LEXC44\n',
 'SI

# 3. Retornando o total comprado e vendido

Com nossa lista de linhas válidas, vamos gerar os valores comprados e vendidos por cada um dos ativos. Primeiro, vamos gerar uma lista com todos os ativos válidos:

In [19]:
ticker = []

In [20]:
for item in validas:
  # Cada string termina em '\n', portanto precisamos recuperar até o penúltimo caracter [:-1]:
  t = item.split(';')[2].split(':')[-1][:-1]
  if t not in ticker:
    ticker.append(t)

In [21]:
ticker

['ACEC4', 'WAYN3', 'LEXC0', 'QEEN36', 'ACEC43', 'ACME11', 'PALM9', 'LEXC44']

Com essa nova lista, estamos prontos para gerar os resultados. Iterando sobre ambas as listas, vamos somar ou subtrair os valores `QTY` da lista `validas` ao `total` se o valor `TICKER` for igual ao elemento da lista `ticker`. Depois, resetamos o `total` e continuamos a iterar até o fim das listas. Vamos então armazenar o nome e o valor de cada ativo em uma nova lista: `ativos`.

In [22]:
ativos = []

In [23]:
# Iterando sobre ambas as listas:
for item in ticker:
  total = 0
  for i in validas:
    # Separando os valores SIDE, QTY e TICKER da lista validas:
    t = i.split(';')[2].split(':')[-1][:-1]
    s = i.split(';')[0].split(':')[-1]
    q = int(i.split(';')[1].split(':')[-1])
    # Comparando TICKER com o elemento da lista ticker:
    if t == item:
      # Somando se o valor for BUY, subtraindo caso SELL:
      if s == 'BUY':
        total += q
      else:
        total -= q
  # Gerando uma string com o nome e valor para incluir na lista:
  a = ''
  x,y,z = str(item),':',str(total)
  a += (x+y+z)
  ativos.append(a)

In [24]:
ativos

['ACEC4:480',
 'WAYN3:4320',
 'LEXC0:4900',
 'QEEN36:-2000',
 'ACEC43:400',
 'ACME11:7650',
 'PALM9:600',
 'LEXC44:-1200']

# 4. Classificando linhas inválidas

Para concluir, vamos repetir os passos da seção 2 para definir a razão de cada linha para ser inválida:

In [25]:
invalidas

['SIDE:B;QTY:100;TICKER:QEEN36\n',
 'SIDE:B;QTY:100;TICKER:WAYNE\n',
 'SIDE:S;QTY:-100;TICKER:ACE4\n',
 'SIDE:S;QTY:100;TICKER:WAYNE\n',
 'SIDE:S;QTY:100;TICKER:LEXC0\n',
 'SIDE:B;QTY:81;TICKER:ACEC4\n',
 'SIDE:B;QTY:100;TICKER:LEX4\n',
 'SIDE:SELL;QTY:369;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:2;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:-100;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:9;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:1;TICKER:ACEC4\n',
 'SIDE:SELL;QTY:-100;TICKER:ACE4\n',
 'SIDE:SELL;QTY:SELL;TICKER:ACE4\n',
 'SIDE:BUY;QTY:33;TICKER:LEXC0\n',
 'SIDE:BUY;QTY:-1;TICKER:ACEC4\n',
 'SIDE:BUY;QTY:95;TICKER:QEEN36\n',
 'SIDE:BUY;QTY:-100;TICKER:ACECHEMICAL\n',
 'SIDE:BUY;QTY:42;TICKER:ACE43\n',
 'SIDE:BUY;QTY:100;TICKER:ACE4\n',
 'SIDE:BUY;QTY:100;TICKER:ACECHEMICAL\n',
 'SIDE:BUY;QTY:100;TICKER:DAILYPLANET\n',
 'SIDE:BUY;QTY:100;TICKER:LEX4\n',
 'SIDE:BUY;QTY:100;TICKER:WAY3\n',
 'SIDE:BUY;QTY:200;TICKER:ACE43\n',
 'SIDE:SELL;QTY:100;TICKER:WAYNE\n',
 'SIDE:BUY;QTY:100;TICKER:WAYNE\n']

In [26]:
ativos_invalidos = []

In [27]:
for item in invalidas:
  l = str(item)
  # Valor inválido de SIDE (B ou S):
  if not re.search(r'\w{3,}', item.split(';')[0].split(':')[-1]):
    l += 'Valor invalido de SIDE; '
  # QTY não múltiplo de 10:
  if not re.search(r'0$', item.split(';')[1].split(':')[-1]):
    l += 'QTY não é múltiplo de 10; '
  # QTY negativo:
  if re.search(r'^-', item.split(';')[1].split(':')[-1]):
    l += 'QTY não é positivo; '
  # TICKER mal formatado:
  if not re.search(r'\D{4,}\d+', item.split(';')[2].split(':')[-1]):
    l += 'TICKER mal formatado; '
  ativos_invalidos.append(l)
  

In [28]:
ativos_invalidos

['SIDE:B;QTY:100;TICKER:QEEN36\nValor invalido de SIDE; ',
 'SIDE:B;QTY:100;TICKER:WAYNE\nValor invalido de SIDE; TICKER mal formatado; ',
 'SIDE:S;QTY:-100;TICKER:ACE4\nValor invalido de SIDE; QTY não é positivo; TICKER mal formatado; ',
 'SIDE:S;QTY:100;TICKER:WAYNE\nValor invalido de SIDE; TICKER mal formatado; ',
 'SIDE:S;QTY:100;TICKER:LEXC0\nValor invalido de SIDE; ',
 'SIDE:B;QTY:81;TICKER:ACEC4\nValor invalido de SIDE; QTY não é múltiplo de 10; ',
 'SIDE:B;QTY:100;TICKER:LEX4\nValor invalido de SIDE; TICKER mal formatado; ',
 'SIDE:SELL;QTY:369;TICKER:ACEC4\nQTY não é múltiplo de 10; ',
 'SIDE:SELL;QTY:2;TICKER:ACEC4\nQTY não é múltiplo de 10; ',
 'SIDE:BUY;QTY:-100;TICKER:ACEC4\nQTY não é positivo; ',
 'SIDE:SELL;QTY:9;TICKER:ACEC4\nQTY não é múltiplo de 10; ',
 'SIDE:BUY;QTY:1;TICKER:ACEC4\nQTY não é múltiplo de 10; ',
 'SIDE:SELL;QTY:-100;TICKER:ACE4\nQTY não é positivo; TICKER mal formatado; ',
 'SIDE:SELL;QTY:SELL;TICKER:ACE4\nQTY não é múltiplo de 10; TICKER mal formatado

# 5. Conclusão

Finalmente, após filtrar as linhas do arquivo, podemos imprimir nossos resultados:

In [29]:
print('Retornar o total que foi comprado e vendido no dia por ativo (TICKER)\n')
for i in ativos:
  print(i)
print('\n Informar quais linhas são inválidas \n')
for j in ativos_invalidos:
  print(j,'\n')

Retornar o total que foi comprado e vendido no dia por ativo (TICKER)

ACEC4:480
WAYN3:4320
LEXC0:4900
QEEN36:-2000
ACEC43:400
ACME11:7650
PALM9:600
LEXC44:-1200

 Informar quais linhas são inválidas 

SIDE:B;QTY:100;TICKER:QEEN36
Valor invalido de SIDE;  

SIDE:B;QTY:100;TICKER:WAYNE
Valor invalido de SIDE; TICKER mal formatado;  

SIDE:S;QTY:-100;TICKER:ACE4
Valor invalido de SIDE; QTY não é positivo; TICKER mal formatado;  

SIDE:S;QTY:100;TICKER:WAYNE
Valor invalido de SIDE; TICKER mal formatado;  

SIDE:S;QTY:100;TICKER:LEXC0
Valor invalido de SIDE;  

SIDE:B;QTY:81;TICKER:ACEC4
Valor invalido de SIDE; QTY não é múltiplo de 10;  

SIDE:B;QTY:100;TICKER:LEX4
Valor invalido de SIDE; TICKER mal formatado;  

SIDE:SELL;QTY:369;TICKER:ACEC4
QTY não é múltiplo de 10;  

SIDE:SELL;QTY:2;TICKER:ACEC4
QTY não é múltiplo de 10;  

SIDE:BUY;QTY:-100;TICKER:ACEC4
QTY não é positivo;  

SIDE:SELL;QTY:9;TICKER:ACEC4
QTY não é múltiplo de 10;  

SIDE:BUY;QTY:1;TICKER:ACEC4
QTY não é múltiplo de 